In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm import tqdm
import time
import geopandas as gpd
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
from multiprocessing import Pool
import os
import re
import matplotlib.pyplot as plt

import mytools
# import importlib
# importlib.reload(mytools)

# 一、计算覆盖空洞生成图层

# 1、-----------------现网覆盖区域计算---------------------------

### 20191107 工参数据图层

In [ ]:
#导入工参数据
gongcan = pd.read_csv('G:/1-规划/工参/工参数据/ltejizhan20200616.csv',
                      usecols=['小区CGI(*)','所属地区','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','归属区县' ],encoding='gbk')

In [ ]:
#设置变量通用名
print('设置变量通用名')
id = '小区CGI(*)'
lon = 'RRU经度'
lat = 'RRU纬度'
zhishi = '工作频段'
fw = '方位角'

In [ ]:
gongcan22 = gongcan[['小区CGI(*)','所属地区']]

In [ ]:
gongcan22.columns = ['CGI','city']

In [ ]:
# gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
#             (gongcan['RRU经度']<117)&
#             (gongcan['RRU纬度']>28)&
#             (gongcan['RRU纬度']<34)&
#            (gongcan['方位角'].notnull())]

# #只要宏站
# gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
# gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('W-')]
# gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('M-')]

# #打标场景
# gongcan = mytools.gisn.points_coverage_merge(gongcan,lon,lat)

# print('添加覆盖距离')
# gongcan_loc_juli = mytools.gisn.changjing_distance(gongcan,'区域类型',zhishi)
# gongcan_loc_juli['distance2'] = gongcan_loc_juli['distance']+gongcan_loc_juli['distance']

In [ ]:
gongcan_loc_juli = pd.read_csv('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/修正覆盖半径后.csv',encoding='gbk')

In [ ]:
gongcan_loc_juli.head(1)

In [ ]:
#做扇区
print('做扇区')
gongcan_s = mytools.gisn.add_sectors_df(gongcan_loc_juli,coords=[lon, lat, 'd_height', fw, 'distance_use_3'], has_z=False, sec_col='geometry', shape_dict={'beam': 120, 'per_degree': 10})
gongcan_s.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/全网覆盖区域_修正覆盖半径后.data')

In [ ]:
gongcan_s=pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/全网覆盖区域_修正覆盖半径后.data')

In [ ]:
# gongcan_s1 = gongcan_s[['小区CGI(*)', '小区中文名', '天线挂高', '方位角',
#        '工作频段', '归属区县', '区域类型', 'distance', 'distance_use_3', 'geometry']]

# gongcan_s1.columns = ['CGI', 'NAME', 'height', 'azimuth', 'frequency', 'county', 'scene', 'distance',
#        'distance_use', 'geometry']

# gongcan_s2 = gongcan_s1.merge(gongcan22,how='left',on='CGI')

# gongcan_s3 = gongcan_s2.loc[gongcan_s2['city'] == '武汉']

# gongcan_s3.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/全网覆盖区域_修正覆盖半径后_武汉.shp',encoding='utf-8')

In [ ]:
gongcan_s = gpd.GeoDataFrame(gongcan_s,crs={'init': 'epsg:4326'},geometry='geometry')

In [ ]:
gongcan_s.plot(figsize=(11,11))

# 2、-----------------求原始覆盖空洞---------------------------

### 1-按照区县合并覆盖区域

In [ ]:
gongcan_s['归属区县'] = gongcan_s['归属区县'].fillna('wu')
gongcan_city = gongcan_s.dissolve(by = '归属区县')
gongcan_city.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/全网覆盖区域_按照区县合并_修正半径后.data')

In [ ]:
gongcan_city = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/全网覆盖区域_按照区县合并.data')

### 2-求出覆盖空洞原始

In [ ]:
tu = mytools.gisn.maps('地市区县')

In [ ]:
tu_mianji = mytools.gisn.add_area(tu)

In [ ]:
tu_mianji[tu_mianji['地市']=='武汉'].to_clipboard()

In [ ]:
tu = mytools.gisn.maps('地市区县')
city_tu_diff = gpd.overlay(tu,gongcan_city,how='difference')
city_tu_diff.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞1-原始_修正半径后.data')

In [ ]:
city_tu_diff = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞1-原始.data')

In [ ]:
tu.plot(figsize=(11,11),color='gray')

In [ ]:
city_tu_diff.plot(figsize=(11,11),color='coral')

# 3、-------------剔除水系-------------

### 1-图中去湖泊

In [ ]:
shuiku = mytools.othern.file_name_paths(path='G:\\1-规划\\图层\\HB_场景\\',find='湖泊、水库.tab')
shuixi2_hb = gpd.read_file(shuiku[0],encoding='gbk')
shuixi2_hb['fenlei']='湖泊'
shuixi2_hb = shuixi2_hb[[ 'geometry', 'fenlei']]
kongd = city_tu_diff.copy()
kongdong_new = gpd.overlay(kongd,shuixi2_hb,how='difference')
kongdong_new.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞2去除湖泊_修正半径后.data')

In [ ]:
kongdong_new = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞2去除湖泊.data')

In [ ]:
shuixi2_hb.plot(figsize=(11,11),color='royalblue')

In [ ]:
shuixi1.plot(figsize=(11,11),color='darkturquoise')

### 2-图中去面状水系

In [ ]:
mianzhuangshuixi =mytools.othern.file_name_paths(path='G:/1-规划/图层/HB_场景/',find='面状水系.tab')
shuixi1 = gpd.read_file(mianzhuangshuixi[0],encoding='gbk')

In [ ]:
shuixi1['fenlei']='面状'

In [ ]:
shuixi1['type'] =  shuixi1.type
shuixi1_use = shuixi1.loc[shuixi1['type']=='Polygon'].append(shuixi1.loc[shuixi1['type']=='MultiPolygon'].explode())
shuixi1_use_area = mytools.gisn.add_area(shuixi1_use)
shuixi1_use_area_use = shuixi1_use_area.loc[shuixi1_use_area['面积']>100]
shuixi1_use_area_use = shuixi1_use_area_use.reset_index(drop=True)
shuixi1_use_area_use['geometry'] = shuixi1_use_area_use.buffer(0.00000001)

In [ ]:
shuixi1_use_area_use.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞2去除湖泊_预备去水系.data')

In [3]:
shuixi1_use_area_use = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞2去除湖泊_预备去水系.data')

In [4]:
def myYield(df):
    y = 1000
    x = 0
    while x<236973:
        shuixi1_use_t = df.iloc[x:y,]
        yield shuixi1_use_t
        print('完成{cishu}。{cishu2}。。'.format(cishu=x,cishu2 = y))
        x = y 
        y = y + 1000

In [5]:
kongdong_new = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-20200706程序输出/2-覆盖空洞-2去除湖泊.data')

In [6]:
tu_new_t = kongdong_new.copy()

In [7]:
for data_t in myYield(shuixi1_use_area_use):
    data_t = data_t.dissolve(by='fenlei')
    tu_new_t = gpd.overlay(tu_new_t,data_t,how='difference')

完成0。1000。。
完成1000。2000。。
完成2000。3000。。
完成3000。4000。。
完成4000。5000。。
完成5000。6000。。
完成6000。7000。。
完成7000。8000。。
完成8000。9000。。
完成9000。10000。。
完成10000。11000。。
完成11000。12000。。
完成12000。13000。。
完成13000。14000。。
完成14000。15000。。
完成15000。16000。。
完成16000。17000。。
完成17000。18000。。
完成18000。19000。。
完成19000。20000。。
完成20000。21000。。
完成21000。22000。。
完成22000。23000。。
完成23000。24000。。
完成24000。25000。。
完成25000。26000。。
完成26000。27000。。
完成27000。28000。。
完成28000。29000。。
完成29000。30000。。
完成30000。31000。。
完成31000。32000。。
完成32000。33000。。
完成33000。34000。。
完成34000。35000。。
完成35000。36000。。
完成36000。37000。。
完成37000。38000。。
完成38000。39000。。
完成39000。40000。。
完成40000。41000。。
完成41000。42000。。
完成42000。43000。。
完成43000。44000。。
完成44000。45000。。
完成45000。46000。。
完成46000。47000。。
完成47000。48000。。
完成48000。49000。。
完成49000。50000。。
完成50000。51000。。
完成51000。52000。。
完成52000。53000。。
完成53000。54000。。
完成54000。55000。。
完成55000。56000。。
完成56000。57000。。
完成57000。58000。。
完成58000。59000。。
完成59000。60000。。
完成60000。61000。。
完成61000。62000。。
完成62000。63000。。
完成63000。64000。

In [8]:
tu_new_t.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-覆盖空洞-2去除湖泊去水系.data')

In [9]:
tu_new_t.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/2-覆盖空洞-2去除湖泊去水系.shp',encoding='utf-8')

In [ ]:
tu_new_t = pd.read_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞2去除湖泊_去水系_修正半径后-完成.data')

In [ ]:
tu_new_t.plot(figsize=(11,11),color='coral')

# 4、-------------空洞有效数据整理-------------

### 1-拆分成多个空洞

In [10]:
tu_new_t['type'] =  tu_new_t.type

In [11]:
tu_new_t.groupby('type').count()

,地市,区县,geometry
type,,,
MultiPolygon,105,105,105
Polygon,1,1,1


In [12]:
kongd_use = tu_new_t.loc[tu_new_t['type']=='Polygon'].append(tu_new_t.loc[tu_new_t['type']=='MultiPolygon'].explode())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [13]:
kongd_mianji = mytools.gisn.add_area(kongd_use)

In [14]:
kongd_mianji.shape

(22502, 5)

In [15]:
kongd_mianji.head(1)

,geometry,type,区县,地市,面积
51,"POLYGON ((114.89187 30.23042, 114.89085 30.229...",Polygon,铁山区,黄石,13154.723419


In [ ]:
kongd_mianji.drop(columns='geometry').to_csv('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞1去除水系的原始空洞_修正半径后空洞21706.csv',index=False,encoding='gbk')

In [ ]:
kongd_mianji.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞1去除水系的原始空洞_修正半径后.shp',encoding='utf-8')

In [ ]:
# 修改算法后这一步不要了


# #最小覆盖半径是250 最极限站间距为覆盖的1.5倍，所以半径为125的园为最小的单位
# kongd_use_youxiao = kongd_mianji.loc[kongd_mianji['面积']>70686]

# #添加id编号
# kongd_use_youxiao2 = kongd_use_youxiao.reset_index(drop=True)
# kongd_use_youxiao2['id'] = [ 'kd_{}'.format(str(name)) for name in kongd_use_youxiao2.index+1]

# # #边简易化
# # kongd_use3 = kongd_use2.copy()
# # kongd_use3['geometry'] = kongd_use3.simplify(tolerance=0.001)

# kongd_use_youxiao2.shape

# kongd_use_youxiao2.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞{}个_修正半径后.shp'.format(
#     kongd_use_youxiao2.shape[0]),encoding='utf-8')

# kongd_use_youxiao2 = gpd.read_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞9819个.shp',encoding='utf-8')

# kongd_use_youxiao2.to_excel('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞7143个_修正半径后.xlsx')

# kongd_use_youxiao2.columns = ['geometry', 'type', 'County', 'city', 'area', 'id']

# kongd_use_youxiao2.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞7143个_修正半径后mapinfo.tab',
#                            driver='MapInfo File',encoding='gbk')

# kongd_use_youxiao2 =  gpd.read_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞7920个_修正半径后mapinfo.tab',encoding='utf-8')

In [ ]:
kongd_mianji.head(1)

In [ ]:
kongd_mianji.plot(figsize=(11,11))

# 有效空洞匹配场景

In [ ]:
# kongd_valid  = gpd.read_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞7143个_修正半径后.shp',encoding='utf-8')

In [ ]:
# kongd_valid.shape

In [16]:
kongd_mianji.crs

{'init': 'epsg:4326'}

In [17]:
kongd_mianji[['lon','lat']] = kongd_mianji.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [18]:
result = mytools.gisn.points_coverage_merge(kongd_mianji,'lon','lat')

In [19]:
result.shape

(22502, 8)

In [20]:
#添加id编号
result_id = result.reset_index(drop=True)
result['id'] = [ 'kd_{}'.format(str(name)) for name in result_id.index+1]

In [21]:
result['制式'] = 'FDD1800'
result.loc[result['区域类型']=='农村','制式']='FDD900'

In [22]:
result = mytools.gisn.changjing_distance(result,'区域类型','制式')

In [23]:
import numpy as np

In [24]:
result['标准面积'] = np.pi* result['distance']* result['distance']

result['空洞面积比例'] = result['面积']/result['标准面积']

In [25]:
def fuck_data(x):
    if x['空洞面积比例']>=1:
        return '大'
    elif x['空洞面积比例']>=1/3:
        return '中'
    elif x['空洞面积比例']>=1/6:
        return '小'
    else:
        return '微'

In [26]:
result['空洞级别'] = result.apply(fuck_data,axis=1)

In [27]:
result.shape

(22502, 14)

In [28]:
result.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/3-覆盖空洞-级别-输出22502个.data')

In [ ]:
result.reindex(columns=['id','地市','区县','面积','lon','lat', '区域类型','制式','distance','标准面积','空洞面积比例','空洞级别']).to_excel(
    'G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3_修正半径后_剔除水系_场景空洞21706个.xlsx',index=False)

In [ ]:
# result = pd.read_excel('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3_修正半径后_剔除水系_场景空洞21706个.xlsx')

In [29]:
result.groupby('空洞级别').count()

,geometry,type,区县,地市,面积,lon,lat,区域类型,id,制式,distance,标准面积,空洞面积比例
空洞级别,,,,,,,,,,,,,
中,761,761,761,761,761,761,761,761,761,761,761,761,761
大,524,524,524,524,524,524,524,524,524,524,524,524,524
小,823,823,823,823,823,823,823,823,823,823,823,823,823
微,20394,20394,20394,20394,20394,20394,20394,20394,20394,20394,20394,20394,20394


In [ ]:
result.columns = ['geometry', 'type', 'county', 'city', 'area', 'lon', 'lat', 'scene', 'NAME', 'frequency',
       'distance', 'stipulate_area', 'proportion', 'level']

In [ ]:
for name,data_t in result.groupby('level'):
    print(name,data_t.shape)
    data_t.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞4_修后_剔水_{}.shp'.format(
    name),encoding='utf-8')

In [ ]:
result.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞5_修后_剔水_21706.shp',encoding='utf-8')

In [ ]:
result.to_pickle('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞5_修后_剔水_21706.data')

# 匹配武汉规划点和空洞的拟合程度

In [ ]:
#导入工参数据
gongcan = pd.read_csv('G:/1-规划/工参/工参数据/ltejizhan20200616.csv',
                      usecols=['小区CGI(*)','网络制式(TDD/FDD)','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','厂家名称','归属区县' ],encoding='gbk')

# gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
#             (gongcan['RRU经度']<117)&
#             (gongcan['RRU纬度']>28)&
#             (gongcan['RRU纬度']<34)]

In [ ]:
gongcan.to_clipboard()

In [ ]:
gongcan.to_csv('d:/gongcan202006016.csv',encoding='gbk')

In [ ]:
data=pd.read_clipboard()

In [ ]:
data_p=mytools.gisn.add_points(data)

In [ ]:
res=gpd.sjoin(kongd_use_youxiao2,data_p)

In [ ]:
res.head(1)

In [ ]:
res.drop_duplicates('id_right').drop(columns='geometry').to_clipboard()

In [ ]:
data.shape

In [ ]:
13/310

# 三角函数计算站点级别的空洞

In [ ]:
duiying = pd.read_csv(open(
    'G:/1-规划/评估/竞对站点评估/20200617/1-基础数据/1-移动数据/2-整理站址数据/1-使用谢总程序跑站点/4-输出/移动站址汇总.csv',
    encoding='gbk'))

In [ ]:
duiying.columns

In [ ]:
duiying['优先级']='TDD'
duiying.loc[duiying['E'].notna(),'优先级']=5
duiying.loc[duiying['D'].notna(),'优先级']=4
duiying.loc[duiying['F'].notna(),'优先级']=3
duiying.loc[duiying['FDD1800'].notna(),'优先级']=2
duiying.loc[duiying['FDD900'].notna(),'优先级']=1

In [ ]:
duiying_sort = duiying.sort_values('优先级')

# 站高和周边的建筑物比较

In [ ]:
duiying = pd.read_csv(open(
    'G:/1-规划/评估/竞对站点评估/20200617/1-基础数据/1-移动数据/2-整理站址数据/1-使用谢总程序跑站点/4-输出/移动站址汇总.csv',
    encoding='gbk'))

In [ ]:
duiying.loc[duiying['FDD900'].notna(),'FDD900']='FDD900'

In [ ]:
duiying_changjing = mytools.gisn.points_coverage_merge(duiying,'CellLongitude','CellLatitude')

In [ ]:
duiying_juli = mytools.gisn.changjing_distance(duiying_changjing,'区域类型','FDD900')

In [ ]:
df_use_jianzhu_buff = mytools.gisn.add_buffer_df(duiying_juli,lon='CellLongitude',lat='CellLatitude',buff_col='distance')

In [ ]:
f = mytools.othern.file_name_paths('G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/',find='tab')

In [ ]:
sjoin_data = gpd.GeoDataFrame()
for name in f:
    tuceng_t = gpd.read_file(name,encoding = 'gbk')
    tuceng_t = tuceng_t.to_crs({'init': 'epsg:4326'})
    linshi_t = gpd.sjoin(df_use_jianzhu_buff,tuceng_t)
    linshi_t = linshi_t[[ '小区中文名','建筑物高度']]
    sjoin_data = sjoin_data.append(linshi_t)

In [ ]:
sjoin_data['建筑物高度2'] = sjoin_data['建筑物高度'] 
sjoin_data['建筑物高度3'] = sjoin_data['建筑物高度'] 

In [ ]:
sjoin_data_linshi = sjoin_data.groupby(by = '小区中文名').agg({'建筑物高度':'mean','建筑物高度2':'count','建筑物高度3':'max'})

In [ ]:
sjoin_data_linshi = sjoin_data_linshi.reset_index()

In [ ]:
df_use_ls6 = duiying.merge(sjoin_data_linshi,how='left',on='小区中文名')

In [ ]:
gongcan = pd.read_csv('G:/1-规划/工参/工参数据/ltejizhan20200616.csv',
                      usecols=['小区CGI(*)','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','归属区县' ]
                      ,encoding='gbk')

In [ ]:
gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
            (gongcan['RRU经度']<117)&
            (gongcan['RRU纬度']>28)&
            (gongcan['RRU纬度']<34)&
           (gongcan['天线挂高'].notnull())]

In [ ]:
gongcan_use = gongcan[['小区中文名','天线挂高']]

In [ ]:
df_use_ls6.drop_duplicates

In [ ]:
kd_jz_gd = df_use_ls6.reset_index().merge(gongcan_use,on='小区中文名').drop_duplicates('index')

In [ ]:
kd_jz_gd.to_csv('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/周边建筑高度/站址级别周边建筑高度输出表.csv',encoding='gbk',index=False)

In [ ]:
# df_use_ls6.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/结果表.csv',encoding='gbk',index=False)

In [ ]:
# df_use_ls6.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/结果表.csv',encoding='gbk',index=False)

# 20191226评估覆盖空洞

## 找出全省覆盖空洞-->设置邮件及-->输出资源分配方案

### 1、找出覆盖空洞 - 空洞的用之前算的1万多
### 主要针对非空洞的进行计算，分为两个方面。1）由于站高原因导致的覆盖优先；2）由于周边建筑高度导致的覆盖优先

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191107.csv',encoding = 'gbk'))

In [ ]:
coverage_hole_cell = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/3-跑出的空洞小区总表-各地市/有空洞的小区-小于0.5.csv',encoding='gbk')) 

In [ ]:
coverage_hole_cell.columns

In [ ]:
data_use = gongcan.merge(coverage_hole_cell,how='outer',left_on='小区CGI(*)',right_on='cgi',indicator=True)

In [ ]:
data_use_left = data_use.loc[data_use['_merge']=='left_only']

In [ ]:
data_use_left = data_use_left[['小区CGI(*)_x','所属地区_x','归属区县', '厂家名称', ' 工作频段', '网络制式(TDD/FDD)',
                               'RRU经度_x', 'RRU纬度_x', '天线挂高','方位角_x','机械下倾角', '电子下倾角']]

In [ ]:
changjing = gpd.read_file('g:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',encoding='gbk')

In [ ]:
data_use_left = data_use_left.loc[(data_use_left['RRU经度_x']>0) & (data_use_left['RRU纬度_x']>0)& (data_use_left['方位角_x']>0)]

In [ ]:
data_use_left_p = tools_gis.add_points(data_use_left,'RRU经度_x','RRU纬度_x')

In [ ]:
res_lishi = gpd.sjoin(data_use_left_p,changjing,how='left')

In [ ]:
res_lishi['区域类型'] = res_lishi['区域类型'].replace({'3A景区':'乡镇','4A景区':'乡镇','5A景区':'乡镇','景区':'乡镇','热点':'乡镇'})

In [ ]:
res_lishi.loc[res_lishi['区域类型'].isna(),'区域类型']='农村'

In [ ]:
res_lishi.groupby('区域类型').count()

In [ ]:
data_use_left_use1 = tools_gis.changjing_distance(res_lishi,changjing='区域类型',zhishi=' 工作频段')

In [ ]:
f = file_name_paths(path='G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/', find='.tab', case_sensitive=False)

In [ ]:
data_use_left_use1 = data_use_left_use1.drop(columns='geometry')

In [ ]:
data_use_left_use1_buff = tools_gis.add_sectors_df(data_use_left_use1,
    coords=['RRU经度_x', 'RRU纬度_x', 'd_height', '方位角_x', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
data_use_left_use1_buff = data_use_left_use1_buff.drop(columns='index_right')

In [ ]:
data_add = pd.DataFrame()
for name in f:
    t_tu = gpd.read_file(name)
    t_tu_1 = t_tu.to_crs({"init": "epsg:4326"})
    data_tu_res = gpd.sjoin(data_use_left_use1_buff,t_tu_1)
    data_tu_res_res = data_tu_res[['小区CGI(*)_x', '方位角_x', 'distance', '单位名称', '建筑物高度']]
    data_tu_res_res['平均值'] = data_tu_res_res['建筑物高度']
    data_group = data_tu_res_res.groupby(by = '小区CGI(*)_x').agg({'建筑物高度':'max','平均值':'mean'})
    data_group.reset_index(inplace = True)
    data_add = data_add.append(data_group)
    print(name,t_tu_1.shape,data_tu_res.shape,data_add.shape)

In [ ]:
data_add.to_csv('G:/1-规划/评估/覆盖空洞/输出20191226/lte142168个侠欧青周边站高.csv',encoding='gbk',index=False)

In [ ]:
hb_louyu = gpd.read_fail('g:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/')

# 20191230使用站点进行评估

## __小区和站点的对应关系 

In [ ]:
f = tools_other.file_name_paths(path='G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/', find=None, case_sensitive=False)

In [ ]:
duiying = pd.DataFrame()
for name in f:
    data = pd.read_excel(name,sheet_name='移动物理站',encoding='gbk')
    data_index = pd.read_excel(name,sheet_name='移动小区',encoding='gbk')
    data['对应小区'] = (data['FDD900_2'].map(str) + '/' + data['FDD1800_2'].map(str) + '/' + data['F_2'].map(str)
                          + '/' + data['D_2'].map(str) + '/'+ data['E_2'].map(str) + '/'+ data['A_2'].map(str) )
    data1 = (data.drop('对应小区', axis=1).join(data['对应小区'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('合并')))#列转行，一列分隔符转多行
    data2 = data1.loc[data1['合并']!='nan']
    linshi = data2.reset_index(drop=True)
    data3 = linshi.reset_index()
    data_index1 = data_index[['序号','小区中文名']]
    data_index1['序号'] = pd.to_numeric(data_index1['序号'])
    data3['合并'] =pd.to_numeric(data3['合并'])
    res_t = data3.merge(data_index1,how='left',left_on='合并',right_on='序号').sort_values('index').drop_duplicates('index')
    duiying = duiying.append(res_t)

In [ ]:
duiying.to_csv('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/站点和小区对应关系.csv',encoding='gbk',index=False)

In [ ]:
duiying = pd.read_csv(open('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/站点和小区对应关系.csv',encoding='gbk'))

In [ ]:
duiying.shape

## __站点汇总

In [ ]:
zhandian = pd.DataFrame()
for name in f:
    data = pd.read_excel(name,sheet_name='移动物理站',encoding='gbk')
    zhandian = zhandian.append(data)

In [ ]:
zhandian = zhandian.drop_duplicates('小区中文名')

In [ ]:
# zhandian.to_csv('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/站点汇总.csv',encoding='gbk',index=False)
zhandian = pd.read_csv(open('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/站点汇总.csv',encoding='gbk'))

In [ ]:
zhandian.shape

## __站点求最近三个站点

In [ ]:
zhandian = pd.read_csv(open('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/计算每个地市的宏/站点汇总.csv',encoding='gbk'))

In [ ]:
zhandian.columns

In [ ]:
zhanzhi_zuijin = tools_gis.nearest_site(zhandian,
    id_name_column='小区中文名',
    lon='CellLongitude',
    lat='CellLatitude',
    num_min_results=3,
    Including_itself=False)

## __站点匹配场景_地市_区县

In [ ]:
guihua = gpd.read_file('G:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB')

In [ ]:
zhanzhi_zuijin_p = tools_gis.add_points(zhanzhi_zuijin,'CellLongitude','CellLatitude')

In [ ]:
zhanzhi_changjing = gpd.sjoin(guihua,zhanzhi_zuijin_p)

In [ ]:
zhanzhi_changjing1 = zhanzhi_changjing[['小区中文名','区域类型']] 

In [ ]:
zhanzhi_zuijin_changjing = zhanzhi_zuijin.merge(zhanzhi_changjing1,how='left',left_on='小区中文名',right_on='小区中文名',indicator=True)

In [ ]:
zhanzhi_zuijin_changjing.loc[zhanzhi_zuijin_changjing['_merge']=='left_only','区域类型']= '农村'

In [ ]:
zhanzhi_zuijin_changjing1 = zhanzhi_zuijin_changjing[[ '小区中文名', '网络制式(TDD/FDD)', 'CellLongitude',
       'CellLatitude','FDD900', 'FDD1800', 'F', 'D', 'E', 'A', 'minimum_index_1', 'minimum_index_2',
       'minimum_index_3', 'distance_1', 'distance_2', 'distance_3', 'mean','区域类型']]

In [ ]:
zhanzhi_zuijin_changjing2 = zhanzhi_zuijin_changjing1.reset_index(drop=True)
zhanzhi_zuijin_changjing3 = zhanzhi_zuijin_changjing2.reset_index()

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191212.csv',encoding='gbk'))

In [ ]:
gongcan_use = gongcan[['小区中文名', '所属地区','归属区县', '厂家名称']]

In [ ]:
zhanzhi_zuijin_changjing4 = zhanzhi_zuijin_changjing3.merge(gongcan_use,how='left',on='小区中文名')

In [ ]:
zhanzhi_zuijin_changjing5 = zhanzhi_zuijin_changjing4.drop_duplicates('index')

## __站点制作扇弧形求覆盖空洞

In [ ]:
hb_ditu = gpd.read_file('g:/1-规划/图层/湖北省地市-边界-边界/全省地市边界.TAB',encoding='gbk')

In [ ]:
zhanzhi_zuijin_changjing5.loc[zhanzhi_zuijin_changjing5['FDD900']>0,'zhishi']='FDD900'

zhanzhi_zuijin_changjing5['区域类型'] = zhanzhi_zuijin_changjing5['区域类型'].replace({'3A景区':'乡镇','4A景区':'乡镇','5A景区':'乡镇','景区':'乡镇','热点':'乡镇'})

zhanzhi_zuijin_changjing6 = tools_gis.changjing_distance(zhanzhi_zuijin_changjing5, '区域类型', 'zhishi')
zhanzhi_zuijin_changjing6.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/计算后站址_间距_字段.csv',encoding='gbk',index=False)
zhanzhi_zuijin_changjing6 = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/计算后站址_间距_字段.csv',encoding='gbk'))

In [ ]:
zhan_use = zhanzhi_zuijin_changjing5[['所属地区', '区域类型', '小区中文名','CellLongitude', 'CellLatitude','distance']]

In [ ]:
data_buff = tools_gis.add_buffer_df(zhan_use,
    lon='CellLongitude',
    lat='CellLatitude',
    buff_col='distance')

In [ ]:
data_city = data_buff.dissolve(by = '所属地区')
hb_ditu_diff = gpd.overlay(hb_ditu,data_city,how='difference')

In [ ]:
hb_ditu_diff.plot(figsize=[13,13])

In [ ]:
# hb_ditu_diff.to_file('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/覆盖空洞图层/fugaikongd.she',encoding='utf-8')

In [ ]:
hb_ditu_diff = gpd.read_file('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/覆盖空洞图层/fugaikongd.she',encoding='utf-8')

In [ ]:
# zhan_use.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/覆盖空洞图层/站址数据.csv',encoding='gbk',index=False)

In [ ]:
zhan_use = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/覆盖空洞图层/站址数据.csv',encoding='gbk'))

## __站点制作2倍扇弧形关联度

In [ ]:
zhan_use = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/覆盖空洞图层/站址数据.csv',encoding='gbk'))
zhan_use['distance_2倍'] = zhan_use['distance'] + zhan_use['distance']
zhan_use['fw'] = 0
zhan_use1=zhan_use.copy()
data_buff_2b = gpd.GeoDataFrame()
for city,data_t in zhan_use1.groupby('所属地区'):
    data_buff_t = tools_gis.add_sectors_min_max(
        data_t,coords=[ 'CellLongitude', 'CellLatitude', 'd_height', 'fw','distance', 'distance_2倍'],
        has_z=False,sec_col='geometry',shape_dict={'beam': 360, 'per_degree': 10})
    data_buff_t_mianji = tools_gis.add_area(data_buff_t,column='面积原')
    data_buff_2b = data_buff_2b.append(data_buff_t_mianji)
    print(city,'ok-------------')
data_buff_2b.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/扇弧形图层/ok.csv',encoding='gbk')

In [ ]:
data = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/扇弧形图层/ok.csv',encoding = 'gbk'),nrows=10)

#                 -------------------使用py文件跑

## __站点覆盖能力最强的小区站高和下倾角

In [ ]:
duiying = pd.read_csv(open('G:/1-规划/评估/竞对站点评估/20200617/1-基础数据/1-移动数据/2-整理站址数据/1-使用谢总程序跑站点/4-输出/移动站址汇总.csv',encoding='gbk'))

In [ ]:
duiying.columns

In [ ]:
duiying_guanxi = duiying.reset_index(drop = True)

In [ ]:
# duiying_guanxi = duiying_linshi.drop(columns='index').reset_index()

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20200616.csv',encoding='gbk'))

In [ ]:
gongcan.columns

In [ ]:
gongcan_use = gongcan[['小区中文名', '天线挂高','机械下倾角', '电子下倾角','工作频段']]

In [ ]:
duiying_guanxi.columns

In [ ]:
duiying_guanxi_merge = duiying_guanxi.merge(gongcan_use,how='left',left_on='小区中文名_x',right_on='小区中文名')

In [ ]:
duiying_use_1 = duiying_guanxi_merge.drop_duplicates('index')

In [ ]:
duiying_use_1['覆盖能力'] = duiying_use_1[' 工作频段'].replace({'FDD900':1,'FDD1800':2,'F':3,'D':4,'E':5,'A':6})

In [ ]:
duiying_use_sort = duiying_use_1.sort_values('覆盖能力')

In [ ]:
max_fw_hide = duiying_use_sort.groupby(by='小区中文名_x').head(1)

## __合并新算法站址，py跑出来的数据

In [ ]:
f = tools_other.file_name_paths(path='G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/空洞站址新算法结果分地市/', find=None, case_sensitive=False)

In [ ]:
zhanzhi_mianji = pd.DataFrame()
for name in f:
    data_t = pd.read_csv(open(name,encoding='gbk'))
    zhanzhi_mianji = zhanzhi_mianji.append(data_t)

In [ ]:
zhanzhi_mianji.shape

## __周边建筑高度

In [ ]:
df_use_jianzhu = zhanzhi_zuijin_changjing6.copy()

In [ ]:
df_use_jianzhu_buff = tools_gis.add_buffer_df(df_use_jianzhu,lon='CellLongitude',lat='CellLatitude',buff_col='distance')

In [ ]:
f = tools_other.file_name_paths('G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/',find='tab')

In [ ]:
sjoin_data = gpd.GeoDataFrame()
for name in f:
    tuceng_t = gpd.read_file(name,encoding = 'gbk')
    tuceng_t = tuceng_t.to_crs({'init': 'epsg:4326'})
    linshi_t = gpd.sjoin(df_use_jianzhu_buff,tuceng_t)
    linshi_t = linshi_t[[ '小区中文名','建筑物高度']]
    sjoin_data = sjoin_data.append(linshi_t)

# __清单整理-匹配以上数据

In [ ]:
df_use = zhanzhi_zuijin_changjing6.copy()

In [ ]:
df_use.columns

In [ ]:
df_use['是否空洞'] = (df_use['distance']-df_use['mean'])>0

In [ ]:
zhanzhi_mianji_linshi2 = zhanzhi_mianji[['小区中文名','空洞面积','面积原']]

In [ ]:
df_use_ls2 = df_use.reset_index(drop=True)

In [ ]:
df_use_ls3 = df_use_ls2.drop(columns='index').reset_index()

In [ ]:
df_use_ls3.drop_duplicates

In [ ]:
df_use_ls4 = df_use_ls3.merge(zhanzhi_mianji_linshi2,how='left',on='小区中文名').drop_duplicates('index')

In [ ]:
df_use_ls4['是否覆盖空洞_用新算法_是否空洞'] = df_use_ls4['空洞面积']/df_use_ls4['面积原']>0.5

In [ ]:
max_fw_hide_ls1 = max_fw_hide[['小区中文名','天线挂高','机械下倾角','电子下倾角',' 工作频段']]

In [ ]:
df_use_ls5 = df_use_ls4.merge(max_fw_hide_ls1,how='left',on='小区中文名').drop_duplicates('index')

In [ ]:
sjoin_data['建筑物高度2'] = sjoin_data['建筑物高度'] 
sjoin_data['建筑物高度3'] = sjoin_data['建筑物高度'] 

In [ ]:
sjoin_data_linshi = sjoin_data.groupby(by = '小区中文名').agg({'建筑物高度':'mean','建筑物高度2':'count','建筑物高度3':'max'})

In [ ]:
df_use_ls6 = df_use_ls5.merge(sjoin_data_linshi,how='left',on='小区中文名').drop_duplicates('index')

In [ ]:
df_use_ls6.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/结果表.csv',encoding='gbk',index=False)

# 小区周边的投诉情况

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191212.csv',encoding='gbk'))

In [ ]:
tousu = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/投诉表/广义投诉CI匹配表-截至12月11日更新TO赵曼.csv',encoding='gbk'))

In [ ]:
gongcan_use_1 = gongcan.loc[(gongcan['RRU经度']>0) & (gongcan['RRU纬度']>0)]

In [ ]:
gongan_p = tools_gis.add_points(gongcan_use_1,lon='RRU经度',lat='RRU纬度')

In [ ]:
tousu.columns

In [ ]:
tousu_buff = tools_gis.add_buffer(tousu,'经度','纬度')

In [ ]:
tousu_buff['geometry'][0]

In [ ]:
res = gpd.sjoin(tousu_buff,gongan_p)

In [ ]:
res.columns

In [ ]:
res_use = res[[ '2019年9月投诉量','2019年10月投诉量', '2019年11月投诉量','小区中文名']]

In [ ]:
res_use

In [ ]:
res_use1 = res_use.groupby(by='小区中文名').agg({'2019年9月投诉量':'sum','2019年10月投诉量':'sum','2019年11月投诉量':'sum'})

In [ ]:
res_use1.to_csv('G:/1-规划/评估/覆盖空洞/输出20191230重新统计站点后/投诉表/res.csv',encoding='gbk')

# 课题是用-覆盖空洞图层

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191212.csv',encoding='gbk'))

In [ ]:
gongcan_use_1 = gongcan.loc[(gongcan['RRU经度']>0) & (gongcan['RRU纬度']>0) & (gongcan['方位角']>-1)]

In [ ]:
gongcan_use = gongcan_use_1.copy()
gongcan_use['小区中文名'] = gongcan_use['小区中文名'].fillna('无')

In [ ]:
gongcan_use = gongcan_use.loc[~gongcan_use['小区中文名'].str.contains('W-')]
gongcan_use = gongcan_use.loc[~gongcan_use['小区中文名'].str.contains('M-')]

In [ ]:
hb_dishi = mytools.gisn.maps('湖北')

In [ ]:
gongcan_use_2 = mytools.gisn.points_coverage_merge(gongcan_use_1)

In [ ]:
gongcan_use_3 = mytools.gisn.changjing_distance(gongcan_use_2,'区域类型',' 工作频段')

In [ ]:
gongcan_sq = mytools.gisn.add_sectors_df( gongcan_use_3,
    coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance'],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
gongcan_sq.loc[gongcan_sq['所属地区'].isna(),['所属地区','方位角']]

In [ ]:
gongcan_sq_hebing = gongcan_sq.dissolve(by='所属地区')

In [ ]:
fugaikongd = gpd.overlay(hb_dishi,gongcan_sq_hebing,how='difference')

In [ ]:
gongcan_use_2 = tools_gis.points_coverage_merge(gongcan_use_1)

In [ ]:
gongcan_use_2

# 水系剔除

In [ ]:
mianzhuangshuixi = mytools.othern.file_name_paths(path='G:\\1-规划\\图层\\HB_场景\\',find='面状水系.tab')
shuiku = mytools.othern.file_name_paths(path='G:\\1-规划\\图层\\HB_场景\\',find='湖泊、水库.tab')

shuixi1 = gpd.read_file(mianzhuangshuixi[0],encoding='gbk')
shuixi2 = gpd.read_file(shuiku[0],encoding='gbk')
print('开始导入覆盖空洞')
kongd = gpd.read_file('G:\\1-规划\\评估\\覆盖空洞\\输出20191230重新统计站点后\\覆盖空洞图层\\fugaikongd\\fugaikongd.shp',encoding='utf-8')
print('导入覆盖空洞图层完成')

shuixi1['fenlei']='面状'
shuixi2['fenlei']='湖泊'

kongdong_new = gpd.overlay(kongd,shuixi2,how='difference')

shuixi1['type'] =  shuixi1.type

shuixi1_use = shuixi1.loc[shuixi1['type']=='Polygon'].append(shuixi1.loc[shuixi1['type']=='MultiPolygon'].explode())

shuixi1_use_area = mytools.gisn.add_area(shuixi1_use)

shuixi1_use_area_use = shuixi1_use_area.loc[shuixi1_use_area['面积']>100]

shuixi1_use_area_use = shuixi1_use_area_use.reset_index(drop=True)

shuixi1_use_area_use['geometry'] = shuixi1_use_area_use.buffer(0.00000001)

shuixi1_use_area_use.shape

# data_t = shuixi1_use_area_use.dissolve(by='fenlei')
# kongdong_new_t = gpd.overlay(kongdong_new,data_t,how='difference')

def myYield(df):
    y = 1000
    x = 0
    while x<236973:
        shuixi1_use_t = df.iloc[x:y,]
        yield shuixi1_use_t
        print('完成{cishu}。{cishu2}。。'.format(cishu=x,cishu2 = y))
        x = y 
        y = y + 1000

kongdong_new_t = kongdong_new.copy()
for data_t in myYield(shuixi1_use_area_use):
    data_t = data_t.dissolve(by='fenlei')
    kongdong_new_t = gpd.overlay(kongdong_new_t,data_t,how='difference')

In [ ]:
kongdong_new_t_r = gpd.read_file('G:/1-规划/报告类/2019年课题2个/寻找有价值覆盖空洞方法/基础数据/跑出的空洞小区总表-各地市/',encoding='utf-8')

# 竞对站点

In [ ]:
te_sit = pd.read_csv(open('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/csv/电信汇总.csv',encoding='gbk'))
un_sit = pd.read_csv(open('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/csv/联通汇总.csv',encoding='gbk'))

In [ ]:
te_site_use = mytools.gisn.points_coverage_merge(te_sit,lon='经度',lat='纬度')
un_site_use = mytools.gisn.points_coverage_merge(un_sit,lon='经度',lat='纬度')

In [ ]:
te_site_use['900-1800-2400'] = te_site_use['900-1800-2400'].replace('900M','FDD900M')
un_site_use['900-1800-2400'] = un_site_use['900-1800-2400'].replace('900M','FDD900M')

In [ ]:
te_site_dis = mytools.gisn.changjing_distance(te_site_use,'区域类型','900-1800-2400')
un_site_dis = mytools.gisn.changjing_distance(un_site_use,'区域类型','900-1800-2400')

In [ ]:
te_site_buffer = mytools.gisn.add_buffer_df(te_site_dis,'经度','纬度','distance')
un_site_buffer = mytools.gisn.add_buffer_df(un_site_dis,'经度','纬度','distance')

In [ ]:
te_site_buffer.columns

In [ ]:
te_site_buffer_dissolve = te_site_buffer.dissolve(by='地市')
un_site_buffer_dissolve = un_site_buffer.dissolve(by='地市')

In [ ]:
te_site_kongd = gpd.overlay(kongdong_new_t_r,te_site_buffer_dissolve)
un_site_kongd = gpd.overlay(kongdong_new_t_r,un_site_buffer_dissolve)

In [ ]:
te_kongd = te_site_kongd.explode()
un_kongd = un_site_kongd.explode()

In [ ]:
te_kongd_use = te_kongd[['geometry']]
un_kongd_use = un_kongd[['geometry']]

In [ ]:
te_kongd_use.to_file('e:/te',encoding=("utf-8"))
un_kongd_use.to_file('e:/un',encoding='utf-8')

In [ ]:
te_kongd_use = gpd.read_file('G:/1-规划/报告类/2019年课题2个/寻找有价值覆盖空洞方法/基础数据/te_kongd/')
un_kongd_use = gpd.read_file('G:/1-规划/报告类/2019年课题2个/寻找有价值覆盖空洞方法/基础数据/un_kongd/')